# Tarefa 1 - Análise de Dados

Sua tarefa é encontrar um conjunto de dados de interesse para você, que contenha uma mistura de variáveis categóricas e numéricas. Como orientação, o conjunto de dados deve ter um mínimo de duas variáveis categóricas e três variáveis numéricas (esses critérios mínimos são orientações e não limites rígidos).
Se desejar, você pode utilizar os seguintes sites para encontrar o conjunto de dados:
• Repositório de dados do governo irlandês: https://data.gov.ie/
• Pesquisa de conjunto de dados do Google: https://datasetsearch.research.google.com/
Você deve então usar os métodos abordados neste curso para concluir uma análise e escrever um relatório sobre os dados usando o Python Notebook. A análise dos dados deve envolver o uso de tabelas, resumos numéricos e resumos gráficos dos dados.
Esta parte da sua atribuição será avaliada em termos de:
• Uso de funções apropriadas em Python. [15 pontos]
• Comentando claramente o código no arquivo .ipynb. [5 pontos]
• Produção de resultados claros para o conjunto de dados escolhido, usando medidas estatísticas apropriadas e gráficos apropriados. [15 pontos]
• A qualidade dos gráficos incluídos. [5 pontos]
• Resumir as conclusões da análise de forma apropriada. [30 pontos]


### Library Import

In [1]:
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt


# Preparação dos dados

### Dataset Reading

In [2]:
df1 = pd.read_csv('Plant_1_Generation_Data.csv')
df2 = pd.read_csv('Plant_2_Generation_Data.csv')

In [3]:
df1.info()
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68778 entries, 0 to 68777
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   DATE_TIME    68778 non-null  object 
 1   PLANT_ID     68778 non-null  int64  
 2   SOURCE_KEY   68778 non-null  object 
 3   DC_POWER     68778 non-null  float64
 4   AC_POWER     68778 non-null  float64
 5   DAILY_YIELD  68778 non-null  float64
 6   TOTAL_YIELD  68778 non-null  float64
dtypes: float64(4), int64(1), object(2)
memory usage: 3.7+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67698 entries, 0 to 67697
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   DATE_TIME    67698 non-null  object 
 1   PLANT_ID     67698 non-null  int64  
 2   SOURCE_KEY   67698 non-null  object 
 3   DC_POWER     67698 non-null  float64
 4   AC_POWER     67698 non-null  float64
 5   DAILY_YIELD  67698 non-null  float64
 6   TOT

In [4]:
# Convertendo a coluna 'DATE_TIME' para o tipo datetime
df1['DATE_TIME'] = pd.to_datetime(df1['DATE_TIME'])
df2['DATE_TIME'] = pd.to_datetime(df2['DATE_TIME'])

### Concatenando os dois Conjuntos de Dados

In [5]:
# axis=0 para concatenar ao longo das linhas (empilhar verticalmente)
# ignore_index=True para redefinir os índices do DataFrame resultante
df = pd.concat([df1, df2], axis=0, ignore_index=True)

In [6]:
df.shape

(136476, 7)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136476 entries, 0 to 136475
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   DATE_TIME    136476 non-null  datetime64[ns]
 1   PLANT_ID     136476 non-null  int64         
 2   SOURCE_KEY   136476 non-null  object        
 3   DC_POWER     136476 non-null  float64       
 4   AC_POWER     136476 non-null  float64       
 5   DAILY_YIELD  136476 non-null  float64       
 6   TOTAL_YIELD  136476 non-null  float64       
dtypes: datetime64[ns](1), float64(4), int64(1), object(1)
memory usage: 7.3+ MB


In [8]:
df.describe()

,DATE_TIME,PLANT_ID,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD
count,136476,1.364760e+05,136476.000000,136476.000000,136476.000000,1.364760e+05
mean,2020-06-01 09:23:03.157478144,4.135497e+06,1708.541497,274.803511,3295.433783,3.303821e+08
min,2020-05-15 00:00:00,4.135001e+06,0.000000,0.000000,0.000000,0.000000e+00
25%,2020-05-23 23:00:00,4.135001e+06,0.000000,0.000000,28.321429,6.520020e+06
50%,2020-06-01 18:45:00,4.135001e+06,6.050000,3.506905,2834.803572,7.269333e+06
75%,2020-06-09 21:45:00,4.136001e+06,1155.788333,532.673333,5992.000000,2.826096e+08
max,2020-06-17 23:45:00,4.136001e+06,14471.125000,1410.950000,9873.000000,2.247916e+09
std,NaN,4.999862e+02,3222.181273,380.182569,3035.294425,6.085705e+08


### Verificando dados Nulos

In [9]:
df.isnull().sum()

DATE_TIME      0
PLANT_ID       0
SOURCE_KEY     0
DC_POWER       0
AC_POWER       0
DAILY_YIELD    0
TOTAL_YIELD    0
dtype: int64

### Adequação de variável

### Separação da variavel DATA_TIME em duas variaveis

In [10]:
# Separando a coluna 'DATE_TIME' em duas colunas diferentes
df['DATE'] = df['DATE_TIME'].dt.date
df['TIME'] = df['DATE_TIME'].dt.time

# Exibindo o DataFrame resultante
print(df1)

                DATE_TIME  PLANT_ID       SOURCE_KEY  DC_POWER  AC_POWER  \
0     2020-05-15 00:00:00   4135001  1BY6WEcLGh8j5v7       0.0       0.0   
1     2020-05-15 00:00:00   4135001  1IF53ai7Xc0U56Y       0.0       0.0   
2     2020-05-15 00:00:00   4135001  3PZuoBAID5Wc2HD       0.0       0.0   
3     2020-05-15 00:00:00   4135001  7JYdWkrLSPkdwr4       0.0       0.0   
4     2020-05-15 00:00:00   4135001  McdE0feGgRqW7Ca       0.0       0.0   
...                   ...       ...              ...       ...       ...   
68773 2020-06-17 23:45:00   4135001  uHbuxQJl8lW7ozc       0.0       0.0   
68774 2020-06-17 23:45:00   4135001  wCURE6d3bPkepu2       0.0       0.0   
68775 2020-06-17 23:45:00   4135001  z9Y9gH1T5YWrNuG       0.0       0.0   
68776 2020-06-17 23:45:00   4135001  zBIq5rxdHJRwDNY       0.0       0.0   
68777 2020-06-17 23:45:00   4135001  zVJPv84UY57bAof       0.0       0.0   

       DAILY_YIELD  TOTAL_YIELD  
0            0.000    6259559.0  
1            0.000 

### Apagando coluna DATE_TIME

In [11]:
df = df.drop(columns=['DATE_TIME'])

### Mudança de nome de variavel

In [12]:
df = df.rename(columns={'SOURCE_KEY':'INVERTER'})

### Function imprimir_valores_unicos()
A function that prints the unique values of each variable.

In [13]:
def imprimir_valores_unicos(data_f):
    for coluna in data_f.columns:
        valores_unicos = data_f[coluna].unique()
        print(f"  {coluna} = {valores_unicos}")

imprimir_valores_unicos(df)

  PLANT_ID = [4135001 4136001]
  INVERTER = ['1BY6WEcLGh8j5v7' '1IF53ai7Xc0U56Y' '3PZuoBAID5Wc2HD' '7JYdWkrLSPkdwr4'
 'McdE0feGgRqW7Ca' 'VHMLBKoKgIrUVDU' 'WRmjgnKYAwPKWDb' 'ZnxXDlPa8U1GXgE'
 'ZoEaEvLYb1n2sOq' 'adLQvlD726eNBSB' 'bvBOhCH3iADSZry' 'iCRJl6heRkivqQ3'
 'ih0vzX44oOqAx2f' 'pkci93gMrogZuBj' 'rGa61gmuvPhdLxV' 'sjndEbLyjtCKgGv'
 'uHbuxQJl8lW7ozc' 'wCURE6d3bPkepu2' 'z9Y9gH1T5YWrNuG' 'zBIq5rxdHJRwDNY'
 'zVJPv84UY57bAof' 'YxYtjZvoooNbGkE' '4UPUqMRk7TRMgml' '81aHJ1q11NBPMrL'
 '9kRcWv60rDACzjR' 'Et9kgGMDl729KT4' 'IQ2d7wF4YD8zU1Q' 'LYwnQax7tkwH5Cb'
 'LlT2YUhhzqhg5Sw' 'Mx2yZCDsyf6DPfv' 'NgDl19wMapZy17u' 'PeE6FRyGXUgsRhN'
 'Qf4GUc1pJu5T6c6' 'Quc1TzYxW2pYoWX' 'V94E5Ben1TlhnDV' 'WcxssY2VbP4hApt'
 'mqwcsP2rE7J0TFp' 'oZ35aAeoifZaQzV' 'oZZkBaNadn6DNKz' 'q49J1IKaHRwDQnt'
 'rrq4fwE8jgrTyWY' 'vOuJvMaM2sgwLmb' 'xMbIugepa2P7lBB' 'xoJJ8DcxJEcupym']
  DC_POWER = [ 0.         37.14285714 53.5        ...  3.78        3.58
  3.90714286]
  AC_POWER = [0.         3.58571429 5.1625     ... 3.63333333 3.45

In [14]:
df

,PLANT_ID,INVERTER,DC_POWER,AC_POWER,DAILY_YIELD,TOTAL_YIELD,DATE,TIME
0,4135001,1BY6WEcLGh8j5v7,0.0,0.0,0.0,6259559.0,2020-05-15,00:00:00
1,4135001,1IF53ai7Xc0U56Y,0.0,0.0,0.0,6183645.0,2020-05-15,00:00:00
2,4135001,3PZuoBAID5Wc2HD,0.0,0.0,0.0,6987759.0,2020-05-15,00:00:00
3,4135001,7JYdWkrLSPkdwr4,0.0,0.0,0.0,7602960.0,2020-05-15,00:00:00
4,4135001,McdE0feGgRqW7Ca,0.0,0.0,0.0,7158964.0,2020-05-15,00:00:00
...,...,...,...,...,...,...,...,...
136471,4136001,q49J1IKaHRwDQnt,0.0,0.0,4157.0,520758.0,2020-06-17,23:45:00
136472,4136001,rrq4fwE8jgrTyWY,0.0,0.0,3931.0,121131356.0,2020-06-17,23:45:00
136473,4136001,vOuJvMaM2sgwLmb,0.0,0.0,4322.0,2427691.0,2020-06-17,23:45:00
136474,4136001,xMbIugepa2P7lBB,0.0,0.0,4218.0,106896394.0,2020-06-17,23:45:00


### Duplicate Data

In [15]:
duplicate_rows_df = df[df.duplicated()]
print("number of duplicate rows: " , duplicate_rows_df.shape)

number of duplicate rows:  (0, 8)


# Analise dos dados

### Mean Median and Mode
encontrando a media, a mediana e a moda das variaveis numéricas

In [16]:
# Para calcular a média das variaveis e imprimi-las
print("Mean of DC_POWER:", df['DC_POWER'].mean())
print("Mean of AC_POWER:", df['AC_POWER'].mean())
print("Mean of DAILY_YIELD:", df['DAILY_YIELD'].mean())
print("Mean of TOTAL_YIELD:", df['TOTAL_YIELD'].mean())

Mean of DC_POWER: 1708.541496702347
Mean of AC_POWER: 274.80351054960653
Mean of DAILY_YIELD: 3295.433783295412
Mean of TOTAL_YIELD: 330382090.0684921


In [17]:
# Para calcular a mediana das variaveis e imprimi-las
print("Median of DC_POWER:", df['DC_POWER'].median())
print("Median of AC_POWER:", df['AC_POWER'].median())
print("Median of DAILY_YIELD:", df['DAILY_YIELD'].median())
print("Median of TOTAL_YIELD:", df['TOTAL_YIELD'].median())

Median of DC_POWER: 6.05
Median of AC_POWER: 3.506904761904762
Median of DAILY_YIELD: 2834.8035715
Median of TOTAL_YIELD: 7269333.0


In [18]:
# Para calcular a moda das variaveis e imprimi-las
print("Mode of DC_POWER:", df['DC_POWER'].mode())
print("Mode of AC_POWER:", df['AC_POWER'].mode())
print("Mode of DAILY_YIELD:", df['DAILY_YIELD'].mode())
print("Mode of TOTAL_YIELD:", df['TOTAL_YIELD'].mode())

Mode of DC_POWER: 0    0.0
Name: DC_POWER, dtype: float64
Mode of AC_POWER: 0    0.0
Name: AC_POWER, dtype: float64
Mode of DAILY_YIELD: 0    0.0
Name: DAILY_YIELD, dtype: float64
Mode of TOTAL_YIELD: 0    0.0
Name: TOTAL_YIELD, dtype: float64


In [19]:
# Calculando o desvio padrão das variáveis e imprimí-las
print("Std Deviation of DC_POWER:", df['DC_POWER'].std())
print("Std Deviation of AC_POWER:", df['AC_POWER'].std())
print("Std Deviation of DAILY_YIELD:", df['DAILY_YIELD'].std())
print("Std Deviation of TOTAL_YIELD:", df['TOTAL_YIELD'].std())

Std Deviation of DC_POWER: 3222.1812733562097
Std Deviation of AC_POWER: 380.18256929405555
Std Deviation of DAILY_YIELD: 3035.294425108002
Std Deviation of TOTAL_YIELD: 608570527.4088567
